# <center>Classifier 1: insulator vs. metal</center>
---

In [37]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
pd.set_option('display.max_columns', None)

First, load in the training sets generated in `dataset.ipynb`.

In [3]:
X = pd.read_csv('../data/X_train_c1.csv')
y = pd.read_csv('../data/y_train_c1.csv')

### Inputs: structural and compositional features

In [7]:
X.head(5)

,minimum CN_VoronoiNN,maximum CN_VoronoiNN,range CN_VoronoiNN,minimum neighbor distance variation,maximum neighbor distance variation,mean ordering parameter shell 1,minimum local difference in Number,maximum local difference in Number,minimum local difference in MendeleevNumber,maximum local difference in MendeleevNumber,minimum local difference in MeltingT,maximum local difference in MeltingT,minimum local difference in CovalentRadius,maximum local difference in CovalentRadius,minimum local difference in NsValence,maximum local difference in NsValence,minimum local difference in NdValence,maximum local difference in NdValence,minimum local difference in NfValence,maximum local difference in NfValence,minimum local difference in NValence,maximum local difference in NValence,minimum local difference in NpUnfilled,maximum local difference in NpUnfilled,minimum local difference in NdUnfilled,maximum local difference in NdUnfilled,minimum local difference in NfUnfilled,maximum local difference in NfUnfilled,minimum local difference in NUnfilled,maximum local difference in NUnfilled,minimum local difference in GSvolume_pa,maximum local difference in GSvolume_pa,minimum local difference in GSbandgap,maximum local difference in GSbandgap,minimum local difference in GSmagmom,maximum local difference in GSmagmom,0-norm,2-norm,MagpieData minimum Number,MagpieData mean MendeleevNumber,MagpieData mode MendeleevNumber,MagpieData minimum MeltingT,MagpieData minimum Column,MagpieData mean Column,MagpieData maximum CovalentRadius,MagpieData maximum NsValence,MagpieData minimum NpValence,MagpieData minimum NdValence,MagpieData mean NdValence,MagpieData minimum NfValence,MagpieData mode NfValence,MagpieData minimum NValence,MagpieData mode NsUnfilled,MagpieData mode NpUnfilled,MagpieData minimum NdUnfilled,MagpieData mode NdUnfilled,MagpieData minimum NfUnfilled,MagpieData mode NfUnfilled,MagpieData minimum NUnfilled,MagpieData mean NUnfilled,MagpieData mode NUnfilled,MagpieData minimum GSvolume_pa,MagpieData mean GSvolume_pa,MagpieData minimum GSbandgap,MagpieData minimum GSmagmom,MagpieData mode GSmagmom,MagpieData maximum SpaceGroupNumber,compound possible
0,5.996321,7.026102,1.029781,6.666706e-03,0.053231,1.268686,3.887179,39.682131,9.055532,19.000000,522.681022,1583.987012,44.167018,75.000000,0.0,0.0,0.000000,10.000000,0.0,0.0,2.000000,6.317869,1.841066,2.111064,0.000000,0.0,0.0,0.0,1.841066,2.111064,9.834127,13.785000,0.0,0.773,0.000000e+00,0.000000,4,0.665852,8.0,77.875000,87.0,54.8,2.0,9.687500,144.0,2.0,0.0,0.0,0.312500,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.125000,2.0,9.105,15.814844,0.0,0.0,0.0,227.0,False
1,5.375365,7.887910,2.512545,5.511299e-10,0.189687,0.686505,3.724667,37.000000,16.635737,28.000000,808.438976,2182.200000,51.129379,76.000000,0.0,1.0,0.000000,8.000000,0.0,0.0,2.283373,3.724667,1.352222,2.000000,0.000000,2.0,0.0,0.0,0.931151,1.862334,4.535000,12.836134,0.0,0.000,0.000000e+00,0.000000,3,0.654654,8.0,76.285714,87.0,54.8,2.0,12.000000,142.0,2.0,0.0,0.0,2.285714,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.000000,2.0,9.105,12.370000,0.0,0.0,0.0,225.0,True
2,5.988688,7.462818,1.474130,8.810336e-03,0.075280,1.136379,2.774280,14.000000,17.921325,82.855659,342.711746,1886.200000,70.719317,94.000000,0.0,1.0,0.000000,2.000000,0.0,0.0,2.000000,4.548560,1.774280,2.000000,0.000000,8.0,0.0,0.0,1.000000,6.000000,7.585000,18.582557,0.0,0.000,0.000000e+00,0.000023,4,0.631219,8.0,71.812500,87.0,54.8,1.0,9.062500,166.0,2.0,0.0,0.0,0.125000,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.562500,2.0,9.105,16.007083,0.0,0.0,0.0,229.0,False
3,4.555664,11.508510,6.952846,1.131330e-02,0.300275,1.080347,11.456281,29.763586,34.148900,78.255186,927.671657,1881.568881,68.556567,127.596742,0.0,0.0,0.025261,4.968877,0.0,0.0,1.168597,3.994433,1.376708,1.990758,0.062398,8.0,0.0,0.0,2.030425,6.022070,3.016144,44.937900,0.0,0.000,2.846023e-07,0.000310,5,0.621130,8.0,59.888889,87.0,54.8,2.0,10.305556,195.0,2.0,0.0,0.0,0.527778,0.0,0.0,2.0,0.0,2.0,0.0,0.0,

### Output: insulator or metal

In [14]:
y.value_counts()

insulator
1            38637
0            26722
dtype: int64

### Training

In [25]:
def classify(model_type, par_dict, X_train, y_train):
    '''
    The skeleton used for training a classifier.
    INPUTS: 
        model_type -- A scikit-learn estimator object.
        par_dict -- The dict of parameters used for CV.
        X_train -- the input of the training set.
        y_train -- the ouput of the training set.
    OUTPUT:
        A trained scikit-learn model.
    '''
    rscv = RandomizedSearchCV(
            estimator = model_type, 
            param_distributions = par_dict, 
            scoring="neg_mean_squared_error",
            verbose = 2, 
            random_state = 42, 
            n_jobs = -1
    )
    rscv.fit(X_train, y_train.values.ravel())
    print(rscv.best_params_)
    model = rscv.best_estimator_
    model.fit(X_train, y_train.values.ravel())
    return model

Let's consider both linear and non-linear models.

In [38]:
choices = [
    [SVC(), {"C": np.logspace(-1, 5, 10), "gamma": np.logspace(-5, 5, 10), "kernel":['poly', 'rbf', 'sigmoid']}],
    [RandomForestClassifier(), {'n_estimators': [int(x) for x in np.linspace(200, 1000, 5)] + [1500, 2000], 'max_depth': [int(x) for x in np.linspace(5, 55, 11)] + [None], 'max_features': ['auto', 'sqrt', 'log2'], 'criterion': ['gini', 'entropy'], 'min_samples_split': [int(x) for x in np.linspace(2, 10, 9)], 'min_impurity_decrease': [0.0, 0.05, 0.1]}],
    [RidgeClassifier(), {"alpha": np.logspace(-6, 3, 10), "class_weight": [None, "balanced"]}],
    [MLPClassifier(), {'hidden_layer_sizes':[(10,10,10),(15,15,15),(20,20)], "alpha": [0.00001,0.0001,0.001,0.01], "activation": ["identity","logistic", "tanh", "relu"]}],
    [LogisticRegression(), {"C": [0.001, 0.01, 0.1, 1, 10, 100], "class_weight": [None, 'balanced'], "penalty": ['l1', 'l2', 'elasticnet', 'none']}],
    [KNeighborsClassifier(), {"n_neighbors": list(range(1,20,1)), "weights": ["uniform", "distance"], "p": [1, 2, 3]}]
]

The next cell trains all models and writes them to `pickle` files in the `jar`. Of course, with a dataset of this size, running it locally would take forever. Hence, I trained all of the models on HPC clusters.

In [ ]:
for choice in choices:
    filename = str(choice[0])[:-2] + '_trained.p'
    trained_model = classify(choice[0], choice[1], X, y)
    pickle.dump(trained_model, open('../jar/classifier1/' + filename, 'wb'))